In [1]:
import os
import json
import re
def determine_file_type(filename):
    # Determine the file type based on the filename
    print(filename)
    if (
    (
        "other_unknown_abeta" in filename
        or "abeta_tau_known_abeta" in filename
    or  "abeta_known_tau_unknown_mmse_known_abeta" in filename)
    and "abeta_known_tau_unknown_mmse_known_abeta_rate" not in filename
    and "abeta_tau_known_abeta_rate" not in filename
):
        print("yes")
        return "abeta"
    elif "abeta_tau_known_meta_tau" in filename or "abeta_unknown_tau_known_meta_tau" in filename:
        return "meta_tau"
    elif "abeta_tau_known_mmse" in filename or "other_unknown_mmse" in filename or "abeta_known_tau_unknown_mmse_known_mmse" in filename:
        return "mmse"
    elif "abeta_tau_known_treatment" in filename:
        return "treatment"
    else:
        return None  # If the file does not match any known type

def extract_probabilities(file_path, file_type):
    # Define token sets based on the file type
    if file_type == "abeta":
        tokens = ['ĠClearly', 'ĠWeak', 'ĠIntermediate', 'ĠNegative']
    elif file_type == "meta_tau":
        tokens = ['ĠLow', 'ĠVery', 'ĠSuper', 'ĠHigh'] # Adjust based on your actual token names
    elif file_type == "mmse":
        tokens = ['ĠSlow', 'ĠModerate', 'ĠFast', 'ĠStable'] # Adjust based on your actual token names
    elif file_type == "treatment":
        tokens = ['Ġideal', 'Ġunlikely'] # Adjust based on your actual token names
    else:
        raise ValueError(f"Unsupported file type: {file_type}")

    position_probabilities = {}

    with open(file_path, "r") as json_file:
        data = json.load(json_file)
        print(json_file)
        # Gather positions for all tokens
        all_positions = set()
        for token in tokens:
            token_data = data.get(token, [])
            for entry in token_data:
                position = entry[0]
                all_positions.add(position)
                if position not in position_probabilities:
                    position_probabilities[position] = {}
                position_probabilities[position][token] = entry[1]
    if not all_positions:
        return 0
    # Determine the best position shared across all tokens
    best_position = None
    for position in sorted(all_positions):
        if all(token in position_probabilities[position] for token in tokens) :
            total_probability = sum(position_probabilities[position][token] for token in tokens)
            if total_probability > 0.5:
                best_position = position
                break
    print(best_position)
    # If no common position is found, fall back to the minimum position that has the most tokens covered

    if best_position is None:
        best_position = min(all_positions, key=lambda pos: (-sum(1 for token in tokens if token in position_probabilities[pos]), pos))


    # Collect probabilities for the chosen position, defaulting to 0 if a token is missing
    probabilities = {token: position_probabilities[best_position].get(token, 0.0) for token in tokens}

    # Normalize the probabilities to sum to 1
    total_probability = sum(probabilities.values())
    
    if total_probability == 0:
        # Handle case where no probability is found, set everything to zero
        normalized_probabilities = {token: 0.0 for token in tokens}
    else:
        # Normalize probabilities
        normalized_probabilities = {token: prob / total_probability for token, prob in probabilities.items()}

    return normalized_probabilities
def extract_abeta_index(filename):
    # Extract the index for abeta files, assuming it's the number before "_abeta"
    match = re.search(r'(\d+)', filename)
    if match:
        return int(match.group(1))
    return None

def process_abeta(directory):
    results = {
        "abeta": {},
        "meta_tau": {},
        "mmse": {},
        "treatment": {}
    }

    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            file_path = os.path.join(directory, filename)
            file_type = determine_file_type(filename)
            if file_type:
                probabilities = extract_probabilities(file_path, file_type)
                if probabilities==0:
                    continue
                # Based on the file type, append probabilities to the correct category
                if file_type == "abeta":
                    abeta_index = extract_abeta_index(filename)
                    print(abeta_index)
                    if abeta_index is not None:
                        results["abeta"][abeta_index] = probabilities
                    if abeta_index is None:
                        results["abeta"][abeta_index] ={0,0,0,0}
#                 elif file_type == "ec_tau":
#                     results["ec_tau"].append(probabilities.get('ĠEC', 0))
#                     results["meta_tau"].append(probabilities.get('ĠMeta', 0))
#                 elif file_type == "mmse":
#                     mmse_index = extract_abeta_index(filename)
#                     if mmse_index is not None:
#                         results["mmse"][mmse_index] = probabilities

    return results

def process_mmse(directory):
    results = {
       "abeta": {},
        "meta_tau": {},
        "mmse": {},
        "treatment": {}
    }

    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            file_path = os.path.join(directory, filename)
            file_type = determine_file_type(filename)
            print(file_type)
            if file_type=="mmse":
                probabilities = extract_probabilities(file_path, file_type)
                print(probabilities)
                # Based on the file type, append probabilities to the correct category
                if file_type == "mmse":
                    mmse_index = extract_abeta_index(filename)
                    print(mmse_index)
                    if mmse_index is not None:
                        results["mmse"][mmse_index] = probabilities
            

    return results
# Usage example
directory_path ="D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\"
extracted_probabilities = process_mmse(directory_path)

# Displaying the result
for biomarker, probs in extracted_probabilities.items():
    print(f"Biomarker: {biomarker}, Probabilities: {probs}")


output_0abeta_tau_known_abeta.json
yes
abeta
output_0abeta_tau_known_meta_tau.json
meta_tau
output_0abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_0abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.2405039962138232, 'ĠModerate': 0.21898139598140676, 'ĠFast': 0.03926120164938605, 'ĠStable': 0.5012534061553839}
0
output_0abeta_tau_known_treatment.json
treatment
output_100abeta_tau_known_abeta.json
yes
abeta
output_100abeta_tau_known_meta_tau.json
meta_tau
output_100abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_100abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.25981074342085864, 'ĠModerate': 0.16514575431558945, 'ĠFast': 0.03355139353186203, 'ĠStable': 0.5414921087316898}
100
output_100abeta_tau_known_treatment.json
treatment
output_101abeta_tau_known_abeta.json
yes
ab

<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_138abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.22216724643930297, 'ĠModerate': 0.2364957852168347, 'ĠFast': 0.07100891652839529, 'ĠStable': 0.47032805181546705}
138
output_138abeta_tau_known_treatment.json
treatment
output_139abeta_tau_known_abeta.json
yes
abeta
output_139abeta_tau_known_meta_tau.json
meta_tau
output_139abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_139abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.22997678827031037, 'ĠModerate': 0.19065745250127325, 'ĠFast': 0.05294366256425175, 'ĠStable': 0.5264220966641646}
139
output_139abeta_tau_known_treatment.json
treatment
output_13abeta_tau_known_abeta.json
yes
abeta
output_13abeta_tau_known_meta_tau.json
meta_tau
output_13abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Rese

<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_17abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.2474513957127214, 'ĠModerate': 0.16483815689443368, 'ĠFast': 0.04722692267866072, 'ĠStable': 0.5404835247141843}
17
output_17abeta_tau_known_treatment.json
treatment
output_180abeta_tau_known_abeta.json
yes
abeta
output_180abeta_tau_known_meta_tau.json
meta_tau
output_180abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_180abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.16775390944909546, 'ĠModerate': 0.47788640170567936, 'ĠFast': 0.19921268976718898, 'ĠStable': 0.15514699907803622}
180
output_180abeta_tau_known_treatment.json
treatment
output_181abeta_tau_known_abeta.json
yes
abeta
output_181abeta_tau_known_meta_tau.json
meta_tau
output_181abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Rese

3
{'ĠSlow': 0.21448798025319663, 'ĠModerate': 0.26279614882488644, 'ĠFast': 0.05423105631330831, 'ĠStable': 0.4684848146086086}
220
output_220abeta_tau_known_treatment.json
treatment
output_221abeta_tau_known_abeta.json
yes
abeta
output_221abeta_tau_known_meta_tau.json
meta_tau
output_221abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_221abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.25938736278231816, 'ĠModerate': 0.38334962335156053, 'ĠFast': 0.08964186523224822, 'ĠStable': 0.2676211486338731}
221
output_221abeta_tau_known_treatment.json
treatment
output_222abeta_tau_known_abeta.json
yes
abeta
output_222abeta_tau_known_meta_tau.json
meta_tau
output_222abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_222abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.1439755733652729

3
{'ĠSlow': 0.26659938969528185, 'ĠModerate': 0.41291720541348914, 'ĠFast': 0.08521027605111753, 'ĠStable': 0.23527312884011153}
25
output_25abeta_tau_known_treatment.json
treatment
output_260abeta_tau_known_abeta.json
yes
abeta
output_260abeta_tau_known_meta_tau.json
meta_tau
output_260abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_260abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.3298077730894644, 'ĠModerate': 0.25685455122140827, 'ĠFast': 0.07833620865649495, 'ĠStable': 0.33500146703263234}
260
output_260abeta_tau_known_treatment.json
treatment
output_261abeta_tau_known_abeta.json
yes
abeta
output_261abeta_tau_known_meta_tau.json
meta_tau
output_261abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_261abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.3066489524004243,

...

3
{'ĠSlow': 0.13920737291137153, 'ĠModerate': 0.3965648987310101, 'ĠFast': 0.1505190543469668, 'ĠStable': 0.3137086740106516}
32
output_32abeta_tau_known_treatment.json
treatment
output_330abeta_tau_known_abeta.json
yes
abeta
output_330abeta_tau_known_meta_tau.json
meta_tau
output_330abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_330abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.32183372922963954, 'ĠModerate': 0.27961397140267136, 'ĠFast': 0.08662036847831098, 'ĠStable': 0.31193193088937815}
330
output_330abeta_tau_known_treatment.json
treatment
output_331abeta_tau_known_abeta.json
yes
abeta
output_331abeta_tau_known_meta_tau.json
meta_tau
output_331abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_331abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.20850777365719306, 

3
{'ĠSlow': 0.29336078891082595, 'ĠModerate': 0.42021991723868773, 'ĠFast': 0.06149175902224569, 'ĠStable': 0.22492753482824063}
364
output_364abeta_tau_known_treatment.json
treatment
output_365abeta_tau_known_abeta.json
yes
abeta
output_365abeta_tau_known_meta_tau.json
meta_tau
output_365abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_365abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.28903386192683883, 'ĠModerate': 0.16727997628923244, 'ĠFast': 0.05964551075239948, 'ĠStable': 0.4840406510315292}
365
output_365abeta_tau_known_treatment.json
treatment
output_366abeta_tau_known_abeta.json
yes
abeta
output_366abeta_tau_known_meta_tau.json
meta_tau
output_366abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_366abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.127820632076160

<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_399abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.23484234666124235, 'ĠModerate': 0.1745199880988612, 'ĠFast': 0.036014240732024014, 'ĠStable': 0.5546234245078724}
399
output_399abeta_tau_known_treatment.json
treatment
output_39abeta_tau_known_abeta.json
yes
abeta
output_39abeta_tau_known_meta_tau.json
meta_tau
output_39abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_39abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.2563309834579009, 'ĠModerate': 0.39701319035426696, 'ĠFast': 0.094298900655506, 'ĠStable': 0.25235692553232614}
39
output_39abeta_tau_known_treatment.json
treatment
output_3abeta_tau_known_abeta.json
yes
abeta
output_3abeta_tau_known_meta_tau.json
meta_tau
output_3abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\A

<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_44abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.18994904872526508, 'ĠModerate': 0.21190345631522117, 'ĠFast': 0.05703304159622687, 'ĠStable': 0.5411144533632869}
44
output_44abeta_tau_known_treatment.json
treatment
output_45abeta_tau_known_abeta.json
yes
abeta
output_45abeta_tau_known_meta_tau.json
meta_tau
output_45abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_45abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.18154538338658327, 'ĠModerate': 0.2294948660703527, 'ĠFast': 0.055368214006455155, 'ĠStable': 0.533591536536609}
45
output_45abeta_tau_known_treatment.json
treatment
output_46abeta_tau_known_abeta.json
yes
abeta
output_46abeta_tau_known_meta_tau.json
meta_tau
output_46abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\

3
{'ĠSlow': 0.1393047655324723, 'ĠModerate': 0.5016557217208119, 'ĠFast': 0.22611396618869659, 'ĠStable': 0.13292554655801922}
80
output_80abeta_tau_known_treatment.json
treatment
output_81abeta_tau_known_abeta.json
yes
abeta
output_81abeta_tau_known_meta_tau.json
meta_tau
output_81abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_81abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.24982060660603456, 'ĠModerate': 0.1587956429177728, 'ĠFast': 0.03713251790357629, 'ĠStable': 0.5542512325726163}
81
output_81abeta_tau_known_treatment.json
treatment
output_82abeta_tau_known_abeta.json
yes
abeta
output_82abeta_tau_known_meta_tau.json
meta_tau
output_82abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_82abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.24937756947739514, 'ĠModerate'

In [89]:
import pandas as pd
# Filter out integer values and keep only dictionary entries
filtered_data = {k: v for k, v in extracted_probabilities['mmse'].items() if isinstance(v, dict)}

# Convert to DataFrame
df = pd.DataFrame.from_dict(filtered_data, orient='index')

# Rename the columns to match the requested format
df.columns = [
    "MMSE-Slow", "MMSE-Moderate", "MMSE-Fast", "MMSE-Not decreasing"
]

print(df)


     MMSE-Slow  MMSE-Moderate  MMSE-Fast  MMSE-Not decreasing
0     0.240504       0.218981   0.039261             0.501253
100   0.259811       0.165146   0.033551             0.541492
101   0.190306       0.130795   0.024963             0.653936
102   0.191039       0.110565   0.021102             0.677294
103   0.245187       0.197013   0.054709             0.503091
..         ...            ...        ...                  ...
96    0.228397       0.156975   0.031397             0.583232
97    0.225898       0.208921   0.048096             0.517085
98    0.333251       0.349244   0.069853             0.247651
99    0.139148       0.396395   0.165242             0.299215
9     0.200609       0.265764   0.067196             0.466431

[440 rows x 4 columns]


In [90]:
df = df.sort_index(key=lambda x: x.astype(int))

In [91]:
import pandas as pd

# Assuming df is the DataFrame already created from the earlier steps

# Step 1: Identify the missing indices in the original DataFrame
expected_indices = set(range(440))
current_indices = set(df.index)
missing_indices = sorted(list(expected_indices - current_indices))

print(f"Missing indices: {missing_indices}")

# Step 2: Reindex the DataFrame to include all indices from 0 to 2159
# Fill missing indices with 0
df_full = df.reindex(range(440), fill_value=0)

# Step 3: Save the updated DataFrame to a CSV file
csv_path_full = 'D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-results\\A4_probability_MMSE.csv'
df_full.to_csv(csv_path_full, index_label='Index')

print(f"CSV file saved at: {csv_path_full}")


Missing indices: []
CSV file saved at: D:\Research\AD\ADLLM\ADNI process 9\A4\A4-results\A4_probability_MMSE.csv


In [92]:
directory_path ="D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\"
extracted_probabilities = process_abeta(directory_path)

# Displaying the result
for biomarker, probs in extracted_probabilities.items():
    print(f"Biomarker: {biomarker}, Probabilities: {probs}")

output_0abeta_tau_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_0abeta_tau_known_abeta.json' mode='r' encoding='cp936'>
5
0
output_0abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_0abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
output_0abeta_tau_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_0abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
output_0abeta_tau_known_treatment.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_0abeta_tau_known_treatment.json' mode='r' encoding='cp936'>
2
output_100abeta_tau_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_100abeta_tau_known_abeta.json

3
output_238abeta_tau_known_treatment.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_238abeta_tau_known_treatment.json' mode='r' encoding='cp936'>
2
output_239abeta_tau_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_239abeta_tau_known_abeta.json' mode='r' encoding='cp936'>
5
239
output_239abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_239abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
output_239abeta_tau_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_239abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
output_239abeta_tau_known_treatment.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_239abeta_

5
output_378abeta_tau_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_378abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
output_378abeta_tau_known_treatment.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_378abeta_tau_known_treatment.json' mode='r' encoding='cp936'>
2
output_379abeta_tau_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_379abeta_tau_known_abeta.json' mode='r' encoding='cp936'>
5
379
output_379abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_379abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
output_379abeta_tau_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_379abeta_tau_k

In [93]:

import pandas as pd
df = pd.DataFrame.from_dict(extracted_probabilities['abeta'], orient='index')

# Rename the columns to match the requested format
df.columns = [
  "Abeta-Clearly positive","Abeta-Weakly positive","Abeta-Intermediate negative","Abeta-Negative"


]
df = df.sort_index(key=lambda x: x.astype(int))

In [94]:
import pandas as pd

# Assuming df is the DataFrame already created from the earlier steps

# Step 1: Identify the missing indices in the original DataFrame
expected_indices = set(range(440))
current_indices = set(df.index)
missing_indices = sorted(list(expected_indices - current_indices))

print(f"Missing indices: {missing_indices}")

# Step 2: Reindex the DataFrame to include all indices from 0 to 2159
# Fill missing indices with 0
df_full = df.reindex(range(440), fill_value=0)

# Step 3: Save the updated DataFrame to a CSV file
csv_path_full = 'D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-results\\A4_probability_abeta.csv'
df_full.to_csv(csv_path_full, index_label='Index')

print(f"CSV file saved at: {csv_path_full}")


Missing indices: []
CSV file saved at: D:\Research\AD\ADLLM\ADNI process 9\A4\A4-results\A4_probability_abeta.csv


In [2]:
def process_tau(directory):
    results = {
        "abeta": {},
        "meta_tau": {},
        "mmse":{}
    }

    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            file_path = os.path.join(directory, filename)
            file_type = determine_file_type(filename)
            print(file_path)
            if file_type == "meta_tau":
                probabilities = extract_probabilities(file_path, file_type)
                print(probabilities)
                if probabilities==0:
                    continue
                if not probabilities:
                    continue
                # Based on the file type, append probabilities to the correct category
                if file_type == "meta_tau":
                    meta_tau_index = extract_abeta_index(filename)
                    print(meta_tau_index)
                    if meta_tau_index is not None:
                        results["meta_tau"][meta_tau_index] = probabilities

    return results

In [95]:
directory_path ="D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\"
extracted_probabilities = process_tau(directory_path)

# Displaying the result
for biomarker, probs in extracted_probabilities.items():
    print(f"Biomarker: {biomarker}, Probabilities: {probs}")

output_0abeta_tau_known_abeta.json
yes
D:\Research\AD\ADLLM\ADNI process 9\A4\A4-inference\A4\probability\output_0abeta_tau_known_abeta.json
output_0abeta_tau_known_meta_tau.json
D:\Research\AD\ADLLM\ADNI process 9\A4\A4-inference\A4\probability\output_0abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_0abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
{'ĠLow': 0.2813354534989732, 'ĠVery': 0.294837021878776, 'ĠSuper': 0.08447222057039235, 'ĠHigh': 0.33935530405185843}
0
output_0abeta_tau_known_mmse.json
D:\Research\AD\ADLLM\ADNI process 9\A4\A4-inference\A4\probability\output_0abeta_tau_known_mmse.json
output_0abeta_tau_known_treatment.json
D:\Research\AD\ADLLM\ADNI process 9\A4\A4-inference\A4\probability\output_0abeta_tau_known_treatment.json
output_100abeta_tau_known_abeta.json
yes
D:\Research\AD\ADLLM\ADNI process 9\A4\A4-inference\A4\probability\output_100abeta_tau_known_abeta.json
ou

<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_30abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
{'ĠLow': 0.1103107085758979, 'ĠVery': 0.12696684845215384, 'ĠSuper': 0.5779867645408483, 'ĠHigh': 0.18473567843109998}
30
output_30abeta_tau_known_mmse.json
D:\Research\AD\ADLLM\ADNI process 9\A4\A4-inference\A4\probability\output_30abeta_tau_known_mmse.json
output_30abeta_tau_known_treatment.json
D:\Research\AD\ADLLM\ADNI process 9\A4\A4-inference\A4\probability\output_30abeta_tau_known_treatment.json
output_310abeta_tau_known_abeta.json
yes
D:\Research\AD\ADLLM\ADNI process 9\A4\A4-inference\A4\probability\output_310abeta_tau_known_abeta.json
output_310abeta_tau_known_meta_tau.json
D:\Research\AD\ADLLM\ADNI process 9\A4\A4-inference\A4\probability\output_310abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_310abeta_tau_known_meta_

In [96]:
import pandas as pd
df_meta_tau = pd.DataFrame.from_dict(extracted_probabilities['meta_tau'], orient='index')

# Rename the columns to match the requested format
df_meta_tau .columns = [
  "Meta Tau-Low","Meta Tau-Very low","Meta Tau-Super high","Meta Tau-High"

]

In [97]:
df_meta_tau = df_meta_tau.sort_index(key=lambda x: x.astype(int))

In [98]:
import pandas as pd

# Assuming df is the DataFrame already created from the earlier steps

# Step 1: Identify the missing indices in the original DataFrame
expected_indices = set(range(440))
current_indices = set(df_meta_tau.index)
missing_indices = sorted(list(expected_indices - current_indices))

print(f"Missing indices: {missing_indices}")

# Step 2: Reindex the DataFrame to include all indices from 0 to 2159
# Fill missing indices with 0
df_full = df_meta_tau.reindex(range(440), fill_value=0)

# Step 3: Save the updated DataFrame to a CSV file
csv_path_full = 'D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-results\\A4_probability_meta_tau.csv'
df_full.to_csv(csv_path_full, index_label='Index')

print(f"CSV file saved at: {csv_path_full}")


Missing indices: []
CSV file saved at: D:\Research\AD\ADLLM\ADNI process 9\A4\A4-results\A4_probability_meta_tau.csv


In [3]:
def process_treatment(directory):
    results = {
        "abeta": {},
        "meta_tau": {},
        "treatment":{}
    }

    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            file_path = os.path.join(directory, filename)
            file_type = determine_file_type(filename)
            print(file_path)
            if file_type == "treatment":
                probabilities = extract_probabilities(file_path, file_type)
                print(probabilities)
                if probabilities==0:
                    continue
                if not probabilities:
                    continue
                # Based on the file type, append probabilities to the correct category
                if file_type == "treatment":
                    meta_tau_index = extract_abeta_index(filename)
                    print(meta_tau_index)
                    if meta_tau_index is not None:
                        results["treatment"][meta_tau_index] = probabilities

    return results

In [99]:
directory_path = "D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\"
extracted_probabilities = process_treatment(directory_path)

# Displaying the result
for biomarker, probs in extracted_probabilities.items():
    print(f"Biomarker: {biomarker}, Probabilities: {probs}")

output_0abeta_tau_known_abeta.json
yes
D:\Research\AD\ADLLM\ADNI process 9\A4\A4-inference\A4\probability\output_0abeta_tau_known_abeta.json
output_0abeta_tau_known_meta_tau.json
D:\Research\AD\ADLLM\ADNI process 9\A4\A4-inference\A4\probability\output_0abeta_tau_known_meta_tau.json
output_0abeta_tau_known_mmse.json
D:\Research\AD\ADLLM\ADNI process 9\A4\A4-inference\A4\probability\output_0abeta_tau_known_mmse.json
output_0abeta_tau_known_treatment.json
D:\Research\AD\ADLLM\ADNI process 9\A4\A4-inference\A4\probability\output_0abeta_tau_known_treatment.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_0abeta_tau_known_treatment.json' mode='r' encoding='cp936'>
2
{'Ġideal': 7.411279979480857e-06, 'Ġunlikely': 0.9999925887200205}
0
output_100abeta_tau_known_abeta.json
yes
D:\Research\AD\ADLLM\ADNI process 9\A4\A4-inference\A4\probability\output_100abeta_tau_known_abeta.json
output_100abeta_tau_known_meta_tau.json
D:\Research\

<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_308abeta_tau_known_treatment.json' mode='r' encoding='cp936'>
2
{'Ġideal': 5.638214592083248e-06, 'Ġunlikely': 0.9999943617854079}
308
output_309abeta_tau_known_abeta.json
yes
D:\Research\AD\ADLLM\ADNI process 9\A4\A4-inference\A4\probability\output_309abeta_tau_known_abeta.json
output_309abeta_tau_known_meta_tau.json
D:\Research\AD\ADLLM\ADNI process 9\A4\A4-inference\A4\probability\output_309abeta_tau_known_meta_tau.json
output_309abeta_tau_known_mmse.json
D:\Research\AD\ADLLM\ADNI process 9\A4\A4-inference\A4\probability\output_309abeta_tau_known_mmse.json
output_309abeta_tau_known_treatment.json
D:\Research\AD\ADLLM\ADNI process 9\A4\A4-inference\A4\probability\output_309abeta_tau_known_treatment.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-inference\\A4\\probability\\output_309abeta_tau_known_treatment.json' mode='r' encoding='cp936'>
None
{'

In [100]:
import pandas as pd
df_treatment = pd.DataFrame.from_dict(extracted_probabilities['treatment'], orient='index')

# Rename the columns to match the requested format
df_treatment .columns = [
  "Treatment-Ideal","Treatment-Unlikely"

]

In [101]:
df_treatment = df_treatment.sort_index(key=lambda x: x.astype(int))

In [102]:
import pandas as pd

# Assuming df is the DataFrame already created from the earlier steps

# Step 1: Identify the missing indices in the original DataFrame
expected_indices = set(range(440))
current_indices = set(df_treatment.index)
missing_indices = sorted(list(expected_indices - current_indices))

print(f"Missing indices: {missing_indices}")

# Step 2: Reindex the DataFrame to include all indices from 0 to 2159
# Fill missing indices with 0
df_full = df_treatment.reindex(range(440), fill_value=0)

# Step 3: Save the updated DataFrame to a CSV file
csv_path_full = 'D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4-results\\A4_probability_treatment.csv'
df_full.to_csv(csv_path_full, index_label='Index')

print(f"CSV file saved at: {csv_path_full}")


Missing indices: []
CSV file saved at: D:\Research\AD\ADLLM\ADNI process 9\A4\A4-results\A4_probability_treatment.csv


In [85]:
df_full

,Treatment-Ideal,Treatment-Unlikely
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0
...,...,...
183,0.0,1.0
184,0.0,1.0
185,0.0,1.0
186,0.0,1.0


## Pointer

In [4]:
import re

# Sample input text
# Regular expressions to capture both "Unknown" and specific values
patterns = {
    'Abeta': r'Abeta: (Unknown.|This patient abeta burden is ([\w\s]+).)',
    'Meta tau': r'Meta tau: (Unknown.|This patient Meta tau burden is ([\w\s]+).)',
    'MMSE rate': r'MMSE rate: (Unknown.|The MMSE is ([\w\s]+).)',
    'Treatment': r'Treatment: (Unknown.|This participant is ([\w\s]+) to be a candidate for anti-amyloid treatment.)'
}

# Function to extract the full sentence
def extract_answers(text, patterns):
    results = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            # Return the full sentence as the answer
            results[key] = match.group(1)
        else:
            results[key] = "Unknown"
    return results


In [34]:
import bigjson
import json
abeta_list=[]
meta_list=[]
mmse_list=[]
treatment_list=[]

In [35]:
with open("D:\\Research\\AD\\ADLLM\\ADNI process 9\\POINTER\\pointer_test_Dec27.json", 'rb') as f:

    json_data = bigjson.load(f)
    for i, data in enumerate(json_data):
        dict_data = data.to_python()
        values = extract_answers(dict_data["output"], patterns)
        abeta_status=values["Abeta"]
        meta_status=values["Meta tau"]
        mmse_rate=values["MMSE rate"]
        treatment=values["Treatment"]
        abeta_list.append(abeta_status)
        meta_list.append(meta_status)
        mmse_list.append(mmse_rate)
        treatment_list.append(treatment)

In [36]:
import pandas as pd
true_pointer=pd.DataFrame({"Abeta":abeta_list,"Meta tau":meta_list,"MMSE":mmse_list,"Treatment":treatment_list})

In [37]:
true_pointer.to_csv("D:\\Research\\AD\\ADLLM\\ADNI process 9\\POINTER\\pointer_test_truth.csv",index=False)

## HABS-HD

In [103]:
import bigjson
import json
abeta_list=[]
meta_list=[]
mmse_list=[]
treatment_list=[]

In [104]:
with open("D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\habs_test_Dec27.json", 'rb') as f:

    json_data = bigjson.load(f)
    for i, data in enumerate(json_data):
        dict_data = data.to_python()
        values = extract_answers(dict_data["output"], patterns)
        abeta_status=values["Abeta"]
        meta_status=values["Meta tau"]
        mmse_rate=values["MMSE rate"]
        treatment=values["Treatment"]
        abeta_list.append(abeta_status)
        meta_list.append(meta_status)
        mmse_list.append(mmse_rate)
        treatment_list.append(treatment)
#         visit_time.append(int(dict_data["Visit time"]))

In [105]:
import pandas as pd
true_habs=pd.DataFrame({"Abeta":abeta_list,"Meta tau":meta_list,"MMSE":mmse_list,"Treatment":treatment_list})

In [106]:
true_habs.to_csv("D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\habs_truth_test.csv",index=False)

## ADNI

In [45]:
import bigjson
import json
abeta_list=[]
meta_list=[]
mmse_list=[]
treatment_list=[]

In [46]:
with open("D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\adni_test_Dec27.json", 'rb') as f:

    json_data = bigjson.load(f)
    for i, data in enumerate(json_data):
        dict_data = data.to_python()
        values = extract_answers(dict_data["output"], patterns)
        abeta_status=values["Abeta"]
        meta_status=values["Meta tau"]
        mmse_rate=values["MMSE rate"]
        treatment=values["Treatment"]
        abeta_list.append(abeta_status)
        meta_list.append(meta_status)
        mmse_list.append(mmse_rate)
        treatment_list.append(treatment)

In [47]:
import pandas as pd
true_adni=pd.DataFrame({"Abeta":abeta_list,"Meta tau":meta_list,"MMSE":mmse_list,"Treatment":treatment_list})

In [48]:
true_adni.to_csv("D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\adni_truth_test.csv",index=False)

## A4

In [49]:
import bigjson
import json
abeta_list=[]
meta_list=[]
mmse_list=[]
treatment_list=[]

In [50]:
with open("D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4_validation_Dec27.json", 'rb') as f:

    json_data = bigjson.load(f)
    for i, data in enumerate(json_data):
        dict_data = data.to_python()
        values = extract_answers(dict_data["output"], patterns)
        abeta_status=values["Abeta"]
        meta_status=values["Meta tau"]
        mmse_rate=values["MMSE rate"]
        treatment=values["Treatment"]
        abeta_list.append(abeta_status)
        meta_list.append(meta_status)
        mmse_list.append(mmse_rate)
        treatment_list.append(treatment)

In [52]:
import pandas as pd
true_a4=pd.DataFrame({"Abeta":abeta_list,"Meta tau":meta_list,"MMSE":mmse_list,"Treatment":treatment_list})

In [53]:
true_a4.to_csv("D:\\Research\\AD\\ADLLM\\ADNI process 9\\A4\\A4_truth_test.csv",index=False)

## Incomplete biomarkers

In [111]:
# Usage example
directory_path ="D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\"
extracted_probabilities = process_mmse(directory_path)

# Displaying the result
for biomarker, probs in extracted_probabilities.items():
    print(f"Biomarker: {biomarker}, Probabilities: {probs}")


output_0other_unknown_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_0other_unknown_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.08988155811288664, 'ĠModerate': 0.23313503426580856, 'ĠFast': 0.5420551984242957, 'ĠStable': 0.13492820919700907}
0
output_1000abeta_unknown_tau_known_meta_tau.json
meta_tau
output_1000abeta_unknown_tau_known_mmse.json
None
output_1004other_unknown_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1004other_unknown_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.08247937806569898, 'ĠModerate': 0.27043946646666367, 'ĠFast': 0.5132039118606448, 'ĠStable': 0.13387724360699252}
1004
output_1006other_unknown_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability

<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1999other_unknown_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.04690005709001371, 'ĠModerate': 0.06116920546666746, 'ĠFast': 0.7451934544447237, 'ĠStable': 0.14673728299859512}
1999
output_19other_unknown_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_19other_unknown_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.03215383869834066, 'ĠModerate': 0.19696612273858316, 'ĠFast': 0.6874793315012622, 'ĠStable': 0.08340070706181398}
19
output_2003other_unknown_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_2003other_unknown_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.22852232563721916, 'ĠModerate':

<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_647other_unknown_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.13837097310030638, 'ĠModerate': 0.3941821930708554, 'ĠFast': 0.2428486071937144, 'ĠStable': 0.22459822663512383}
647
output_679other_unknown_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_679other_unknown_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.07204923692460576, 'ĠModerate': 0.13046430511184068, 'ĠFast': 0.395628112559276, 'ĠStable': 0.40185834540427756}
679
output_681other_unknown_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_681other_unknown_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.10066433351393082, 'ĠModerate': 0.

In [112]:
import pandas as pd
# Filter out integer values and keep only dictionary entries
filtered_data = {k: v for k, v in extracted_probabilities['mmse'].items() if isinstance(v, dict)}

# Convert to DataFrame
df = pd.DataFrame.from_dict(filtered_data, orient='index')

# Rename the columns to match the requested format
df.columns = [
    "MMSE-Slow", "MMSE-Moderate", "MMSE-Fast", "MMSE-Not decreasing"
]

print(df)


      MMSE-Slow  MMSE-Moderate  MMSE-Fast  MMSE-Not decreasing
0      0.089882       0.233135   0.542055             0.134928
1004   0.082479       0.270439   0.513204             0.133877
1006   0.038028       0.218836   0.653325             0.089810
1014   0.117722       0.325040   0.200251             0.356987
1016   0.080937       0.424077   0.346122             0.148865
...         ...            ...        ...                  ...
994    0.567807       0.276727   0.084397             0.071069
995    0.508270       0.328164   0.106539             0.057026
996    0.439803       0.413156   0.057690             0.089351
997    0.435827       0.486200   0.038682             0.039291
998    0.071872       0.760720   0.149795             0.017613

[569 rows x 4 columns]


In [113]:
df = df.sort_index(key=lambda x: x.astype(int))

In [114]:
import pandas as pd

# Assuming df is the DataFrame already created from the earlier steps

# Step 1: Identify the missing indices in the original DataFrame
expected_indices = set(range(2464))
current_indices = set(df.index)
missing_indices = sorted(list(expected_indices - current_indices))

print(f"Missing indices: {missing_indices}")

# Step 2: Reindex the DataFrame to include all indices from 0 to 2159
# Fill missing indices with 0
df_full = df.reindex(range(2464), fill_value=0)

# Step 3: Save the updated DataFrame to a CSV file
csv_path_full = 'D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-results\\ADNI_probability_MMSE_incomplete.csv'
df_full.to_csv(csv_path_full, index_label='Index')

print(f"CSV file saved at: {csv_path_full}")


Missing indices: [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 36, 37, 38, 40, 43, 46, 48, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 112, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 167, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 181, 183, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 211, 212, 213, 214, 215, 216, 218, 221, 223, 225, 226, 227, 229, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 244, 245, 246, 247, 248, 249, 250, 251,

In [115]:
# Usage example
directory_path ="D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\"
extracted_probabilities = process_abeta(directory_path)

# Displaying the result
for biomarker, probs in extracted_probabilities.items():
    print(f"Biomarker: {biomarker}, Probabilities: {probs}")


output_0other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_0other_unknown_mmse.json' mode='r' encoding='cp936'>
3
output_1000abeta_unknown_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1000abeta_unknown_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
output_1000abeta_unknown_tau_known_mmse.json
output_1004other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1004other_unknown_mmse.json' mode='r' encoding='cp936'>
3
output_1006other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1006other_unknown_mmse.

5
1120
output_1120abeta_known_tau_unknown_mmse_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1120abeta_known_tau_unknown_mmse_known_mmse.json' mode='r' encoding='cp936'>
3
output_1126abeta_known_tau_unknown_mmse_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1126abeta_known_tau_unknown_mmse_known_abeta.json' mode='r' encoding='cp936'>
5
1126
output_1126abeta_known_tau_unknown_mmse_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1126abeta_known_tau_unknown_mmse_known_mmse.json' mode='r' encoding='cp936'>
3
output_1129abeta_known_tau_unknown_mmse_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADN

<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1220abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
output_1220abeta_tau_known_treatment.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1220abeta_tau_known_treatment.json' mode='r' encoding='cp936'>
2
output_1224abeta_tau_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1224abeta_tau_known_abeta.json' mode='r' encoding='cp936'>
5
1224
output_1224abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1224abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
output_1224abeta_tau_kno

5
1290
output_1290abeta_known_tau_unknown_mmse_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1290abeta_known_tau_unknown_mmse_known_mmse.json' mode='r' encoding='cp936'>
3
output_1292abeta_tau_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1292abeta_tau_known_abeta.json' mode='r' encoding='cp936'>
5
1292
output_1292abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1292abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
output_1292abeta_tau_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1292abeta_ta

3
output_1407abeta_tau_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1407abeta_tau_known_abeta.json' mode='r' encoding='cp936'>
5
1407
output_1407abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1407abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
output_1407abeta_tau_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1407abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
output_1407abeta_tau_known_treatment.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1407abeta_tau_known_treatment.json' mode='r' encod

...

3
output_267other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_267other_unknown_mmse.json' mode='r' encoding='cp936'>
3
output_291other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_291other_unknown_mmse.json' mode='r' encoding='cp936'>
3
output_293other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_293other_unknown_mmse.json' mode='r' encoding='cp936'>
3
output_295other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_295other_unknown_mmse.json' mode='r' encoding='cp936'>
3
output_2other_unknown_mmse.json
<_io.T

3
output_466other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_466other_unknown_mmse.json' mode='r' encoding='cp936'>
3
output_468other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_468other_unknown_mmse.json' mode='r' encoding='cp936'>
3
output_469other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_469other_unknown_mmse.json' mode='r' encoding='cp936'>
3
output_476other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_476other_unknown_mmse.json' mode='r' encoding='cp936'>
3
output_478other_unknown_mmse.json
<_io

2
output_597other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_597other_unknown_mmse.json' mode='r' encoding='cp936'>
3
output_613other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_613other_unknown_mmse.json' mode='r' encoding='cp936'>
3
output_615other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_615other_unknown_mmse.json' mode='r' encoding='cp936'>
3
output_626other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_626other_unknown_mmse.json' mode='r' encoding='cp936'>
3
output_628other_unknown_mmse.json
<_io

5
858
output_858abeta_known_tau_unknown_mmse_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_858abeta_known_tau_unknown_mmse_known_mmse.json' mode='r' encoding='cp936'>
3
output_860abeta_tau_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_860abeta_tau_known_abeta.json' mode='r' encoding='cp936'>
5
860
output_860abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_860abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
output_860abeta_tau_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_860abeta_tau_known_mm

3
output_982other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_982other_unknown_mmse.json' mode='r' encoding='cp936'>
3
output_984other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_984other_unknown_mmse.json' mode='r' encoding='cp936'>
3
output_992other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_992other_unknown_mmse.json' mode='r' encoding='cp936'>
3
output_994other_unknown_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_994other_unknown_mmse.json' mode='r' encoding='cp936'>
3
output_995other_unknown_mmse.json
<_io

In [116]:

import pandas as pd
df = pd.DataFrame.from_dict(extracted_probabilities['abeta'], orient='index')

# Rename the columns to match the requested format
df.columns = [
  "Abeta-Clearly positive","Abeta-Weakly positive","Abeta-Intermediate negative","Abeta-Negative"


]
df = df.sort_index(key=lambda x: x.astype(int))

In [117]:
import pandas as pd

# Assuming df is the DataFrame already created from the earlier steps

# Step 1: Identify the missing indices in the original DataFrame
expected_indices = set(range(2464))
current_indices = set(df.index)
missing_indices = sorted(list(expected_indices - current_indices))

print(f"Missing indices: {missing_indices}")

# Step 2: Reindex the DataFrame to include all indices from 0 to 2159
# Fill missing indices with 0
df_full = df.reindex(range(2464), fill_value=0)

# Step 3: Save the updated DataFrame to a CSV file
csv_path_full = 'D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-results\\ADNI_probability_abeta_incomplete.csv'
df_full.to_csv(csv_path_full, index_label='Index')

print(f"CSV file saved at: {csv_path_full}")


Missing indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 218, 220, 221, 222, 223, 224,

In [118]:
# Usage example
directory_path ="D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\"
extracted_probabilities = process_tau(directory_path)

# Displaying the result
for biomarker, probs in extracted_probabilities.items():
    print(f"Biomarker: {biomarker}, Probabilities: {probs}")


output_0other_unknown_mmse.json
D:\Research\AD\ADLLM\ADNI process 9\ADNI\ADNI-inference\ADNI\probability_incomplete\probability_incomplete\output_0other_unknown_mmse.json
output_1000abeta_unknown_tau_known_meta_tau.json
D:\Research\AD\ADLLM\ADNI process 9\ADNI\ADNI-inference\ADNI\probability_incomplete\probability_incomplete\output_1000abeta_unknown_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\output_1000abeta_unknown_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
{'ĠLow': 0.11333947057312252, 'ĠVery': 0.13045292192622598, 'ĠSuper': 0.593856321208195, 'ĠHigh': 0.16235128629245654}
1000
output_1000abeta_unknown_tau_known_mmse.json
D:\Research\AD\ADLLM\ADNI process 9\ADNI\ADNI-inference\ADNI\probability_incomplete\probability_incomplete\output_1000abeta_unknown_tau_known_mmse.json
output_1004other_unknown_mmse.json
D:\Research\AD\ADLLM\ADNI process 9\ADNI\ADNI-i

In [119]:
import pandas as pd
df_meta_tau = pd.DataFrame.from_dict(extracted_probabilities['meta_tau'], orient='index')

# Rename the columns to match the requested format
df_meta_tau .columns = [
  "Meta Tau-Low","Meta Tau-Very low","Meta Tau-Super high","Meta Tau-High"

]

In [120]:
import pandas as pd

# Assuming df is the DataFrame already created from the earlier steps

# Step 1: Identify the missing indices in the original DataFrame
expected_indices = set(range(2464))
current_indices = set(df_meta_tau.index)
missing_indices = sorted(list(expected_indices - current_indices))

print(f"Missing indices: {missing_indices}")

# Step 2: Reindex the DataFrame to include all indices from 0 to 2159
# Fill missing indices with 0
df_full = df_meta_tau.reindex(range(2464), fill_value=0)

# Step 3: Save the updated DataFrame to a CSV file
csv_path_full = 'D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-results\\ADNI_probability_meta_tau_incomplete.csv'
df_full.to_csv(csv_path_full, index_label='Index')

print(f"CSV file saved at: {csv_path_full}")


Missing indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 21

In [122]:
# Usage example
directory_path ="D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-inference\\ADNI\\probability_incomplete\\probability_incomplete\\"
extracted_probabilities = process_treatment(directory_path)

# Displaying the result
for biomarker, probs in extracted_probabilities.items():
    print(f"Biomarker: {biomarker}, Probabilities: {probs}")


output_0other_unknown_mmse.json
D:\Research\AD\ADLLM\ADNI process 9\ADNI\ADNI-inference\ADNI\probability_incomplete\probability_incomplete\output_0other_unknown_mmse.json
output_1000abeta_unknown_tau_known_meta_tau.json
D:\Research\AD\ADLLM\ADNI process 9\ADNI\ADNI-inference\ADNI\probability_incomplete\probability_incomplete\output_1000abeta_unknown_tau_known_meta_tau.json
output_1000abeta_unknown_tau_known_mmse.json
D:\Research\AD\ADLLM\ADNI process 9\ADNI\ADNI-inference\ADNI\probability_incomplete\probability_incomplete\output_1000abeta_unknown_tau_known_mmse.json
output_1004other_unknown_mmse.json
D:\Research\AD\ADLLM\ADNI process 9\ADNI\ADNI-inference\ADNI\probability_incomplete\probability_incomplete\output_1004other_unknown_mmse.json
output_1006other_unknown_mmse.json
D:\Research\AD\ADLLM\ADNI process 9\ADNI\ADNI-inference\ADNI\probability_incomplete\probability_incomplete\output_1006other_unknown_mmse.json
output_1014other_unknown_mmse.json
D:\Research\AD\ADLLM\ADNI process 9\AD

In [123]:
import pandas as pd
df_treatment = pd.DataFrame.from_dict(extracted_probabilities['treatment'], orient='index')

# Rename the columns to match the requested format
df_treatment .columns = [
  "Treatment-Ideal","Treatment-Unlikely"

]

In [124]:
import pandas as pd

# Assuming df is the DataFrame already created from the earlier steps

# Step 1: Identify the missing indices in the original DataFrame
expected_indices = set(range(2464))
current_indices = set(df_treatment.index)
missing_indices = sorted(list(expected_indices - current_indices))

print(f"Missing indices: {missing_indices}")

# Step 2: Reindex the DataFrame to include all indices from 0 to 2159
# Fill missing indices with 0
df_full = df_treatment.reindex(range(2464), fill_value=0)

# Step 3: Save the updated DataFrame to a CSV file
csv_path_full = 'D:\\Research\\AD\\ADLLM\\ADNI process 9\\ADNI\\ADNI-results\\ADNI_probability_treatment_incomplete.csv'
df_full.to_csv(csv_path_full, index_label='Index')

print(f"CSV file saved at: {csv_path_full}")


Missing indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 21

## Incomplete HABS-HD

In [9]:
# Usage example
directory_path ="D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\"
extracted_probabilities = process_abeta(directory_path)
# Displaying the result
for biomarker, probs in extracted_probabilities.items():
    print(f"Biomarker: {biomarker}, Probabilities: {probs}")





output_0other_unknown_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_0other_unknown_abeta.json' mode='r' encoding='cp936'>
5
0
output_10abeta_tau_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_10abeta_tau_known_abeta.json' mode='r' encoding='cp936'>
5
10
output_10abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_10abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
output_10abeta_tau_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_10abeta_tau_known_mmse.json' mode='

<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_161abeta_tau_known_abeta.json' mode='r' encoding='cp936'>
5
161
output_161abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_161abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
output_161abeta_tau_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_161abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
output_161abeta_tau_known_treatment.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_161abeta_tau_known_treatment.json' mode='r' encoding='cp936'>
Non

<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_174abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
output_174abeta_tau_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_174abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
output_174abeta_tau_known_treatment.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_174abeta_tau_known_treatment.json' mode='r' encoding='cp936'>
None
output_175abeta_tau_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_175abeta_tau_known_abeta.json' mode='r' encoding='cp936'>
5
1

<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_1abeta_tau_known_treatment.json' mode='r' encoding='cp936'>
None
output_20abeta_tau_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_20abeta_tau_known_abeta.json' mode='r' encoding='cp936'>
5
20
output_20abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_20abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
output_20abeta_tau_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_20abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
output_

3
output_27abeta_tau_known_treatment.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_27abeta_tau_known_treatment.json' mode='r' encoding='cp936'>
None
output_28abeta_tau_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_28abeta_tau_known_abeta.json' mode='r' encoding='cp936'>
9
28
output_28abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_28abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
9
output_28abeta_tau_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_28abeta_tau_known_mmse

...

<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_45abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
output_45abeta_tau_known_treatment.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_45abeta_tau_known_treatment.json' mode='r' encoding='cp936'>
None
output_46abeta_tau_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_46abeta_tau_known_abeta.json' mode='r' encoding='cp936'>
5
46
output_46abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_46abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
o

None
output_55abeta_tau_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_55abeta_tau_known_abeta.json' mode='r' encoding='cp936'>
5
55
output_55abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_55abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
output_55abeta_tau_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_55abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
output_55abeta_tau_known_treatment.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_55abeta_tau_known_treatment

9
66
output_66abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_66abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
9
output_66abeta_tau_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_66abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
output_66abeta_tau_known_treatment.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_66abeta_tau_known_treatment.json' mode='r' encoding='cp936'>
None
output_67abeta_tau_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_67abeta_tau_known_abeta

None
output_74other_unknown_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_74other_unknown_abeta.json' mode='r' encoding='cp936'>
5
74
output_75abeta_tau_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_75abeta_tau_known_abeta.json' mode='r' encoding='cp936'>
5
75
output_75abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_75abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
output_75abeta_tau_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_75abeta_tau_known_mmse.json

9
8
output_8abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_8abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
9
output_8abeta_tau_known_mmse.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_8abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
8
output_8abeta_tau_known_treatment.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_8abeta_tau_known_treatment.json' mode='r' encoding='cp936'>
None
output_9abeta_tau_known_abeta.json
yes
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_9abeta_tau_known_abeta.json' mo

In [10]:

import pandas as pd
df = pd.DataFrame.from_dict(extracted_probabilities['abeta'], orient='index')

# Rename the columns to match the requested format
df.columns = [
  "Abeta-Clearly positive","Abeta-Weakly positive","Abeta-Intermediate negative","Abeta-Negative"


]
df = df.sort_index(key=lambda x: x.astype(int))

In [11]:
import pandas as pd

# Assuming df is the DataFrame already created from the earlier steps

# Step 1: Identify the missing indices in the original DataFrame
expected_indices = set(range(2464))
current_indices = set(df.index)
missing_indices = sorted(list(expected_indices - current_indices))

print(f"Missing indices: {missing_indices}")

# Step 2: Reindex the DataFrame to include all indices from 0 to 2159
# Fill missing indices with 0
df_full = df.reindex(range(2464), fill_value=0)

# Step 3: Save the updated DataFrame to a CSV file
csv_path_full = 'D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-results\\HABS_probability_abeta_incomplete.csv'
df_full.to_csv(csv_path_full, index_label='Index')

print(f"CSV file saved at: {csv_path_full}")


Missing indices: [11, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 166, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 214, 215, 216, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 230, 231, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322,

In [12]:
# Usage example
directory_path ="D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\"
extracted_probabilities = process_tau(directory_path)
# Displaying the result
for biomarker, probs in extracted_probabilities.items():
    print(f"Biomarker: {biomarker}, Probabilities: {probs}")





output_0other_unknown_abeta.json
yes
D:\Research\AD\ADLLM\ADNI process 9\HABS-HD\HABS-HD-inference\HABS-HD\probability_incomplete\probability_incomplete\output_0other_unknown_abeta.json
output_10abeta_tau_known_abeta.json
yes
D:\Research\AD\ADLLM\ADNI process 9\HABS-HD\HABS-HD-inference\HABS-HD\probability_incomplete\probability_incomplete\output_10abeta_tau_known_abeta.json
output_10abeta_tau_known_meta_tau.json
D:\Research\AD\ADLLM\ADNI process 9\HABS-HD\HABS-HD-inference\HABS-HD\probability_incomplete\probability_incomplete\output_10abeta_tau_known_meta_tau.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_10abeta_tau_known_meta_tau.json' mode='r' encoding='cp936'>
5
{'ĠLow': 0.2555396236477889, 'ĠVery': 0.32811936208651915, 'ĠSuper': 0.10324733068727261, 'ĠHigh': 0.31309368357841927}
10
output_10abeta_tau_known_mmse.json
D:\Research\AD\ADLLM\ADNI process 9\HABS-HD\HABS-H

In [13]:
import pandas as pd
df_meta_tau = pd.DataFrame.from_dict(extracted_probabilities['meta_tau'], orient='index')

# Rename the columns to match the requested format
df_meta_tau .columns = [
  "Meta Tau-Low","Meta Tau-Very low","Meta Tau-Super high","Meta Tau-High"

]

In [14]:
import pandas as pd

# Assuming df is the DataFrame already created from the earlier steps

# Step 1: Identify the missing indices in the original DataFrame
expected_indices = set(range(2464))
current_indices = set(df_meta_tau.index)
missing_indices = sorted(list(expected_indices - current_indices))

print(f"Missing indices: {missing_indices}")

# Step 2: Reindex the DataFrame to include all indices from 0 to 2159
# Fill missing indices with 0
df_full = df_meta_tau.reindex(range(2464), fill_value=0)

# Step 3: Save the updated DataFrame to a CSV file
csv_path_full = 'D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-results\\HABS_probability_meta_tau_incomplete.csv'
df_full.to_csv(csv_path_full, index_label='Index')

print(f"CSV file saved at: {csv_path_full}")


Missing indices: [0, 5, 11, 13, 15, 18, 19, 21, 25, 29, 30, 40, 47, 52, 56, 58, 61, 64, 65, 68, 74, 77, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 166, 167, 171, 173, 176, 177, 178, 179, 180, 185, 186, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 230, 231, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292,

In [16]:
# Usage example
directory_path ="D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\"
extracted_probabilities = process_mmse(directory_path)
# Displaying the result
for biomarker, probs in extracted_probabilities.items():
    print(f"Biomarker: {biomarker}, Probabilities: {probs}")





output_0other_unknown_abeta.json
yes
abeta
output_10abeta_tau_known_abeta.json
yes
abeta
output_10abeta_tau_known_meta_tau.json
meta_tau
output_10abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_10abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.24696862705982914, 'ĠModerate': 0.2047441868941796, 'ĠFast': 0.04939683462258534, 'ĠStable': 0.49889035142340593}
10
output_10abeta_tau_known_treatment.json
treatment
output_12abeta_tau_known_abeta.json
yes
abeta
output_12abeta_tau_known_meta_tau.json
meta_tau
output_12abeta_tau_known_mmse.json
mmse
<_io.TextIOWrapper name='D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\output_12abeta_tau_known_mmse.json' mode='r' encoding='cp936'>
3
{'ĠSlow': 0.3547132333447291, 'ĠModerate': 0.2554903759714582, 'ĠFast': 0.0910

In [17]:
import pandas as pd
# Filter out integer values and keep only dictionary entries
filtered_data = {k: v for k, v in extracted_probabilities['mmse'].items() if isinstance(v, dict)}

# Convert to DataFrame
df = pd.DataFrame.from_dict(filtered_data, orient='index')

# Rename the columns to match the requested format
df.columns = [
    "MMSE-Slow", "MMSE-Moderate", "MMSE-Fast", "MMSE-Not decreasing"
]

print(df)


     MMSE-Slow  MMSE-Moderate  MMSE-Fast  MMSE-Not decreasing
10    0.246969       0.204744   0.049397             0.498890
12    0.354713       0.255490   0.091098             0.298698
14    0.148168       0.248136   0.132817             0.470879
153   0.284943       0.232564   0.074332             0.408162
154   0.016437       0.077203   0.883539             0.022821
..         ...            ...        ...                  ...
7     0.153957       0.200798   0.116213             0.529032
80    0.215543       0.264088   0.170508             0.349860
82    0.308489       0.162562   0.051152             0.477797
8     0.244271       0.349902   0.304000             0.101827
9     0.153608       0.357149   0.174060             0.315183

[107 rows x 4 columns]


In [18]:
import pandas as pd

# Assuming df is the DataFrame already created from the earlier steps

# Step 1: Identify the missing indices in the original DataFrame
expected_indices = set(range(2464))
current_indices = set(df.index)
missing_indices = sorted(list(expected_indices - current_indices))

print(f"Missing indices: {missing_indices}")

# Step 2: Reindex the DataFrame to include all indices from 0 to 2159
# Fill missing indices with 0
df_full = df.reindex(range(2464), fill_value=0)

# Step 3: Save the updated DataFrame to a CSV file
csv_path_full = 'D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-results\\HABS_probability_mmse_incomplete.csv'
df_full.to_csv(csv_path_full, index_label='Index')

print(f"CSV file saved at: {csv_path_full}")


Missing indices: [0, 5, 11, 13, 15, 18, 19, 21, 25, 29, 30, 40, 47, 52, 56, 58, 61, 64, 65, 68, 74, 77, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 166, 167, 171, 173, 176, 177, 178, 179, 180, 185, 186, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 230, 231, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 256, 257, 258, 259, 260, 261, 262, 263, 265, 266, 267, 268, 270, 272, 273, 274, 276, 278, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299,

In [19]:
# Usage example
directory_path ="D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-inference\\HABS-HD\\probability_incomplete\\probability_incomplete\\"
extracted_probabilities = process_treatment(directory_path)
# Displaying the result
for biomarker, probs in extracted_probabilities.items():
    print(f"Biomarker: {biomarker}, Probabilities: {probs}")





output_0other_unknown_abeta.json
yes
D:\Research\AD\ADLLM\ADNI process 9\HABS-HD\HABS-HD-inference\HABS-HD\probability_incomplete\probability_incomplete\output_0other_unknown_abeta.json
output_10abeta_tau_known_abeta.json
yes
D:\Research\AD\ADLLM\ADNI process 9\HABS-HD\HABS-HD-inference\HABS-HD\probability_incomplete\probability_incomplete\output_10abeta_tau_known_abeta.json
output_10abeta_tau_known_meta_tau.json
D:\Research\AD\ADLLM\ADNI process 9\HABS-HD\HABS-HD-inference\HABS-HD\probability_incomplete\probability_incomplete\output_10abeta_tau_known_meta_tau.json
output_10abeta_tau_known_mmse.json
D:\Research\AD\ADLLM\ADNI process 9\HABS-HD\HABS-HD-inference\HABS-HD\probability_incomplete\probability_incomplete\output_10abeta_tau_known_mmse.json
output_10abeta_tau_known_treatment.json
D:\Research\AD\ADLLM\ADNI process 9\HABS-HD\HABS-HD-inference\HABS-HD\probability_incomplete\probability_incomplete\output_10abeta_tau_known_treatment.json
<_io.TextIOWrapper name='D:\\Research\\AD\\ADL

In [21]:
import pandas as pd
df_treatment = pd.DataFrame.from_dict(extracted_probabilities['treatment'], orient='index')

# Rename the columns to match the requested format
df_treatment .columns = [
  "Treatment-Ideal","Treatment-Unlikely"

]

In [22]:
import pandas as pd

# Assuming df is the DataFrame already created from the earlier steps

# Step 1: Identify the missing indices in the original DataFrame
expected_indices = set(range(2464))
current_indices = set(df_treatment.index)
missing_indices = sorted(list(expected_indices - current_indices))

print(f"Missing indices: {missing_indices}")

# Step 2: Reindex the DataFrame to include all indices from 0 to 2159
# Fill missing indices with 0
df_full = df_treatment.reindex(range(2464), fill_value=0)

# Step 3: Save the updated DataFrame to a CSV file
csv_path_full = 'D:\\Research\\AD\\ADLLM\\ADNI process 9\\HABS-HD\\HABS-HD-results\\HABS_probability_treatment_incomplete.csv'
df_full.to_csv(csv_path_full, index_label='Index')

print(f"CSV file saved at: {csv_path_full}")


Missing indices: [0, 5, 11, 13, 15, 18, 19, 21, 25, 29, 30, 40, 47, 52, 56, 58, 61, 64, 65, 68, 74, 77, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 166, 167, 171, 173, 176, 177, 178, 179, 180, 185, 186, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 230, 231, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292,